# Clustering Crypto

In [1]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [2]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv", index_col=0)
print(crypto_df)

            CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
42           42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
365          365Coin       X11       True   PoW/PoS              NaN   
404          404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
611        SixEleven   SHA-256       True       PoW              NaN   
808              808   SHA-256       True   PoW/PoS     0.000000e+00   
...              ...       ...        ...       ...              ...   
XBC      BitcoinPlus    Scrypt       True       PoS     1.283270e+05   
DVTC      DivotyCoin    Scrypt      False   PoW/PoS     2.149121e+07   
GIOT     Giotto Coin    Scrypt      False   PoW/PoS              NaN   
OPSC  OpenSourceCoin   SHA-256      False   PoW/PoS              NaN   
PUNK       SteamPunk       PoS      False       PoS              NaN   

     TotalCoinSupply  
42                42  
365       2300000000  
404        532000000  
611           611000  
808                0

In [3]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
print(crypto_df)

         CoinName    Algorithm  IsTrading ProofType  TotalCoinsMined  \
42        42 Coin       Scrypt       True   PoW/PoS     4.199995e+01   
365       365Coin          X11       True   PoW/PoS              NaN   
404       404Coin       Scrypt       True   PoW/PoS     1.055185e+09   
611     SixEleven      SHA-256       True       PoW              NaN   
808           808      SHA-256       True   PoW/PoS     0.000000e+00   
...           ...          ...        ...       ...              ...   
SERO   Super Zero       Ethash       True       PoW              NaN   
UOS           UOS      SHA-256       True      DPoI              NaN   
BDX        Beldex  CryptoNight       True       PoW     9.802226e+08   
ZEN       Horizen     Equihash       True       PoW     7.296538e+06   
XBC   BitcoinPlus       Scrypt       True       PoS     1.283270e+05   

     TotalCoinSupply  
42                42  
365       2300000000  
404        532000000  
611           611000  
808                0

In [4]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=['IsTrading'], inplace=True)
print(crypto_df)

         CoinName    Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42        42 Coin       Scrypt   PoW/PoS     4.199995e+01              42
365       365Coin          X11   PoW/PoS              NaN      2300000000
404       404Coin       Scrypt   PoW/PoS     1.055185e+09       532000000
611     SixEleven      SHA-256       PoW              NaN          611000
808           808      SHA-256   PoW/PoS     0.000000e+00               0
...           ...          ...       ...              ...             ...
SERO   Super Zero       Ethash       PoW              NaN      1000000000
UOS           UOS      SHA-256      DPoI              NaN      1000000000
BDX        Beldex  CryptoNight       PoW     9.802226e+08      1400222610
ZEN       Horizen     Equihash       PoW     7.296538e+06        21000000
XBC   BitcoinPlus       Scrypt       PoS     1.283270e+05         1000000

[1144 rows x 5 columns]


In [5]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace=True)
print(crypto_df)

         CoinName    Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42        42 Coin       Scrypt   PoW/PoS     4.199995e+01              42
404       404Coin       Scrypt   PoW/PoS     1.055185e+09       532000000
808           808      SHA-256   PoW/PoS     0.000000e+00               0
1337    EliteCoin          X13   PoW/PoS     2.927942e+10    314159265359
BTC       Bitcoin      SHA-256       PoW     1.792718e+07        21000000
...           ...          ...       ...              ...             ...
ZEPH       ZEPHYR      SHA-256      DPoS     2.000000e+09      2000000000
GAP       Gapcoin       Scrypt   PoW/PoS     1.493105e+07       250000000
BDX        Beldex  CryptoNight       PoW     9.802226e+08      1400222610
ZEN       Horizen     Equihash       PoW     7.296538e+06        21000000
XBC   BitcoinPlus       Scrypt       PoS     1.283270e+05         1000000

[685 rows x 5 columns]


In [6]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
print(crypto_df)

         CoinName    Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42        42 Coin       Scrypt   PoW/PoS     4.199995e+01              42
404       404Coin       Scrypt   PoW/PoS     1.055185e+09       532000000
1337    EliteCoin          X13   PoW/PoS     2.927942e+10    314159265359
BTC       Bitcoin      SHA-256       PoW     1.792718e+07        21000000
ETH      Ethereum       Ethash       PoW     1.076842e+08               0
...           ...          ...       ...              ...             ...
ZEPH       ZEPHYR      SHA-256      DPoS     2.000000e+09      2000000000
GAP       Gapcoin       Scrypt   PoW/PoS     1.493105e+07       250000000
BDX        Beldex  CryptoNight       PoW     9.802226e+08      1400222610
ZEN       Horizen     Equihash       PoW     7.296538e+06        21000000
XBC   BitcoinPlus       Scrypt       PoS     1.283270e+05         1000000

[532 rows x 5 columns]


In [7]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name = crypto_df[['CoinName']].copy()
print(coin_name)

         CoinName
42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
...           ...
ZEPH       ZEPHYR
GAP       Gapcoin
BDX        Beldex
ZEN       Horizen
XBC   BitcoinPlus

[532 rows x 1 columns]


In [8]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=['CoinName'])
print(crypto_df)

        Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42         Scrypt   PoW/PoS     4.199995e+01              42
404        Scrypt   PoW/PoS     1.055185e+09       532000000
1337          X13   PoW/PoS     2.927942e+10    314159265359
BTC       SHA-256       PoW     1.792718e+07        21000000
ETH        Ethash       PoW     1.076842e+08               0
...           ...       ...              ...             ...
ZEPH      SHA-256      DPoS     2.000000e+09      2000000000
GAP        Scrypt   PoW/PoS     1.493105e+07       250000000
BDX   CryptoNight       PoW     9.802226e+08      1400222610
ZEN      Equihash       PoW     7.296538e+06        21000000
XBC        Scrypt       PoS     1.283270e+05         1000000

[532 rows x 4 columns]


In [16]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
404,1.055185e+09,532000000,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1337,2.927942e+10,314159265359,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
BTC,1.792718e+07,21000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ETH,1.076842e+08,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
GAP,1.493105e+07,250000000,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
BDX,9.802226e+08,1400222610,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ZEN,7.296538e+06,21000000,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [18]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.34254594,  1.05112497, -0.50894452],
       [-0.32589962,  1.05137527, -0.50938797],
       [ 2.31005817,  1.62489244, -0.62175664],
       ...,
       [ 0.33165818, -2.32401396,  0.33641261],
       [-0.18797094, -1.97886782,  0.28521317],
       [-0.29064295,  0.86265783, -0.21066138]])

In [19]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2', 'PC3'], index=crypto_df.index)
print(pca_df.shape)
pca_df

(532, 3)


,PC1,PC2,PC3
42,-0.342546,1.051125,-0.508945
404,-0.325900,1.051375,-0.509388
1337,2.310058,1.624892,-0.621757
BTC,-0.142294,-1.331077,0.138318
ETH,-0.148329,-2.059677,0.404981
...,...,...,...
ZEPH,2.483323,0.747347,-0.325793
GAP,-0.340592,1.051017,-0.508971
BDX,0.331658,-2.324014,0.336413
ZEN,-0.187971,-1.978868,0.285213


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [21]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

/Users/miracleonyeoziri/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/miracleonyeoziri/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/miracleonyeoziri/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/miracleonyeoziri/opt/anaconda3/envs/mlenv/lib/python3.11/site-p

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [25]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
predictions

/Users/miracleonyeoziri/opt/anaconda3/envs/mlenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [28]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pca_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coin_name

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.342546,1.051125,-0.508945,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.325900,1.051375,-0.509388,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.310058,1.624892,-0.621757,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.142294,-1.331077,0.138318,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.148329,-2.059677,0.404981,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.172136,-1.095119,0.003901,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.397795,1.226723,-0.460172,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.137914,-2.307586,0.456929,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.146772,-2.059771,0.404961,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.187970,-1.978868,0.285213,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [32]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Class',
                    symbol='Class',
                    hover_data=['Algorithm']
)
fig.show()

In [39]:
# Create a table with tradable cryptocurrencies.
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
table = clustered_df[columns].hvplot.table(sortable=True, selectable=True)
table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [34]:
# Print the total number of tradable cryptocurrencies.
total_cryptos = len(clustered_df)

# Print the total number of tradable cryptocurrencies
print(f"There are {total_cryptos} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [35]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler()
scaled = scaler.fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [40]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
columns = ['TotalCoinSupply', 'TotalCoinsMined']
scaled_df = pd.DataFrame(scaled, columns=columns, index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df['Class'] = clustered_df['Class']

scaled_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [42]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
scaled_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols=["CoinName"]).opts(width=800, height=500)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)